This is a tutorial on how to model in vivo TF binding using sequnce feature, DNA shape features and histone modification (HM) features.

Description of data:
- MYC_pwm.txt : Position weight matrix of MYC binding sites in format of FIMO (Grant et al. 2011) input;
- BS.bed : Coordinates of BSs;
- non-BS.bed : Coordinates of non-BSs;
- BS.txt : Sequences of BSs matched from BS.bed;
- non-BS.txt : Sequeces of non-BSs matched from non-BS.bed;
- BS_10_HM.txt : 10 HMs patterns around BSs, 10 HMs are H3K4me2, H3K27ac, H3K27me3, H3K36me3, H3K4me1, H3K4me3, H3K79me2, H4K20me1, H3K9me1, and H3K9me3, numbers are shown in order;
- non-BS_10_HM.txt : 10 HMs patterns around non-BSs;

The following will explain how to generate these data and how to model TF binding with L2-regularized MLR of a specific TF, in this case, we use MYC.


First, we generate data with following steps: generate BS.bed from ChIP-seq data -> generate non-BS.bed from chromatin accessible regions -> scan 10 HM patterns.
- Download .narrowPeak data from http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeAwgTfbsUniform/;
- Put MYC.narrowPeak under ./MYC ;
- Download genome-wide HM data for 10 HMs in .bam format from http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/ and put them under ./10_histone_modification/, note that some HMs have repetitive experiments, name each data set, for example, h3k9me3_rep1.bam. For each HM dataset, calculate read counts by using ./getCounts.sh, name each count file as h3k9me3_rep1_count.txt
- Download DNase-seq data for each cell type from http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeAwgDnaseUniform/, to get chromatin accessible regions, put data in ./ as DNase_accessible.fa; 
- Download h19 reference sequence from UCSC, put it under ./ as h19.fa; 
- run the following script, then we will get data above data.


In [1]:
#Generate BS.bed, BS_10_HM.txt.
bash ./generate_BS_data.sh ./ MYC 10 1000 0

#Generate non-BS.bed, non-BS_10_HM.txt, 1235 below is the number of sequences in BS.bed. Note that each time non-BS.bed might be different.
bash ./generate_non-BS_data.sh ./ MYC 10 1235 1000 0


Second, calculate significance level of HM pattern differences between BSs and non-BSs. Those numbers are visualized in MYC row in Figure 2A, highlighted by a green rectangle. 


In [2]:
#install qvalue library from Bioconductro if encessary: http://bioconductor.org/packages/release/bioc/html/qvalue.html
Rscript q_value_from_p_value.R ./ MYC

![description](Figure_2A_MYC.png)

Third, generate data as input for L2-regularized MLR models.Data for MLR modeling are stored in MYC/encoded.
- MYC.txt stack non-BSs (first column assigned label 0) and BSs (first column assigned label 1), input of one-hot encoded and DNAshapeR;
- MYC.00000000001 data for HM models. Stack 10 HM patterns around non-BSs and BSs, same order as MYC.txt. First column still shows the label for each sequence, second column shows all 1s, which is intercept term in MLR model;
- MYC.MGW, MYC.Roll, MYC.ProT, and MYC.HelT DNA shape profiles generated by DNAshapeR software (Chiu et al., 2016).
- MYC.10000000000 one-hot encoded data for sequence models; 
- MYC.00011110000 DNA shape data for shape model; 
- MYC.10011110000 data for sequence+shape model; 
- MYC.11000000000 data for sequence+shape+HM model; 
- MYC.10000000001 data for sequence+HM model.

In [3]:
sh ./generate_mlr_data.sh ./ MYC 10


Next, run L2-regularized MLR models, sequence model, sequence+shape model, sequence+shape+HM model, sequence+HM model, HM model. Report model performance in AUROC. Results are visualized in Figure 3.

In [4]:
Rscript shuffle_divide.R MYC ./MYC/input 10 feature_list_mlr 
# use ./train.py to get input arguments.
./train.py MYC ./MYC/encoded ./MYC/input ./MYC/output 10 ./train.R ./feature_list 
./test.py MYC ./MYC/encoded ./MYC/input ./MYC/output ./predict.R ./feature_list 
./summarize_mlr_auroc.py 


![description](Figure_3A_MYC.png)

References:

Chiu T, Comoglio F, Zhou T, Yang L, Paro R, Rohs R. 2016. DNAshapeR : an R / Bioconductor package for DNA shape prediction and feature encoding. Bioinformatics 32: 1211–1213.

Grant CE, Bailey TL, Noble WS. 2011. FIMO: Scanning for occurrences of a given motif. Bioinformatics 27: 1017–1018.
